In [2]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points
num_samples = 100000

# Generate synthetic data
temperature = np.random.uniform(50, 150, num_samples)    # Temperature range: 50-150
pressure = np.random.uniform(10, 30, num_samples)       # Pressure range: 10-30
speed = np.random.uniform(1000, 3000, num_samples)      # Speed range: 1000-3000

# Assuming linear relationship for output and energy
# Output = f(Temperature, Pressure, Speed) + random noise
output = 2 * temperature + 3 * pressure + 0.5 * speed + np.random.normal(0, 10, num_samples)
energy = 0.5 * temperature + 1.5 * pressure + 0.2 * speed + np.random.normal(0, 5, num_samples)

# Create a DataFrame
df = pd.DataFrame({
    'Temperature': temperature,
    'Pressure': pressure,
    'Speed': speed,
    'Output': output,
    'Energy': energy
})

# Save the synthetic data to a CSV file
df.to_csv('equipment_data.csv', index=False)

# Display first few rows of the generated data
print(df.head())


   Temperature   Pressure        Speed       Output      Energy
0    87.454012  21.615581  1565.175934  1021.516381  381.864670
1   145.071431  20.539433  1917.353181  1299.152091  487.255853
2   123.199394  17.020739  1198.430996   893.204779  326.386002
3   109.865848  19.864253  1893.674051  1233.159077  466.351213
4    65.601864  17.301933  1406.162699   881.276156  334.871419


In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('equipment_data.csv')

# Display first few rows and check for any missing values
print(df.head())
print(df.info())


   Temperature   Pressure        Speed       Output      Energy
0    87.454012  21.615581  1565.175934  1021.516381  381.864670
1   145.071431  20.539433  1917.353181  1299.152091  487.255853
2   123.199394  17.020739  1198.430996   893.204779  326.386002
3   109.865848  19.864253  1893.674051  1233.159077  466.351213
4    65.601864  17.301933  1406.162699   881.276156  334.871419
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Temperature  100000 non-null  float64
 1   Pressure     100000 non-null  float64
 2   Speed        100000 non-null  float64
 3   Output       100000 non-null  float64
 4   Energy       100000 non-null  float64
dtypes: float64(5)
memory usage: 3.8 MB
None


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Select features and target
X = df[['Temperature', 'Pressure', 'Speed']]
y = df['Output']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on test data
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} - Mean Squared Error: {mse}')

    # Print feature importances (if applicable)
    if name == 'Decision Tree' or name == 'Random Forest':
        print(f'{name} - Feature Importances:', model.feature_importances_)
    
    print()  # Just for separating model outputs


Linear Regression - Mean Squared Error: 101.00909375504264

Decision Tree - Mean Squared Error: 227.84124930804515
Decision Tree - Feature Importances: [0.03840654 0.00380376 0.9577897 ]

Random Forest - Mean Squared Error: 120.97646266827604
Random Forest - Feature Importances: [0.03832778 0.00374041 0.95793181]



In [10]:
pip install -U bayesian-optimization

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for bayesian-optimization from https://files.pythonhosted.org/packages/af/f6/adfa3c95b1d910f9102e6ed1babbe458497ff7707f540b091dbf66631df9/bayesian_optimization-1.5.0-py3-none-any.whl.metadata


In [11]:
from bayes_opt import BayesianOptimization
import numpy as np

# Define function to maximize output prediction
def predict_output_bayesian(temperature, pressure, speed):
    return -model.predict([[temperature, pressure, speed]])[0]

# Define bounds for Bayesian Optimization
pbounds = {'temperature': (min(df['Temperature']), max(df['Temperature'])),
           'pressure': (min(df['Pressure']), max(df['Pressure'])),
           'speed': (min(df['Speed']), max(df['Speed']))}

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=predict_output_bayesian,
    pbounds=pbounds,
    random_state=42,
)

# Perform optimization
optimizer.maximize(init_points=10, n_iter=50)

# Get the best parameters
best_params = optimizer.max['params']
best_temperature = best_params['temperature']
best_pressure = best_params['pressure']
best_speed = best_params['speed']

# Print optimized settings
print('Bayesian Optimization - Optimized Settings:')
print('Temperature:', best_temperature)
print('Pressure:', best_pressure)
print('Speed:', best_speed)


|   iter    |  target   | pressure  |   speed   | temper... |
-------------------------------------------------------------
| 1         | -1.739e+0 | 17.49     | 2.901e+03 | 123.2     |
| 2         | -849.7    | 21.97     | 1.312e+03 | 65.6      |
| 3         | -1.618e+0 | 11.16     | 2.732e+03 | 110.1     |
| 4         | -888.5    | 24.16     | 1.041e+03 | 147.0     |
| 5         | -928.3    | 26.65     | 1.425e+03 | 68.18     |
| 6         | -1.049e+0 | 13.67     | 1.608e+03 | 102.5     |
| 7         | -1.076e+0 | 18.64     | 1.582e+03 | 111.2     |
| 8         | -1.001e+0 | 12.79     | 1.584e+03 | 86.64     |
| 9         | -1.486e+0 | 19.12     | 2.57e+03  | 69.97     |
| 10        | -1.267e+0 | 20.28     | 2.185e+03 | 54.65     |
| 11        | -643.9    | 10.0      | 1e+03     | 50.0      |


c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not

| 12        | -674.0    | 10.0      | 1.102e+03 | 50.0      |
| 13        | -695.9    | 30.0      | 1e+03     | 50.0      |


c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


| 14        | -656.2    | 11.13     | 1.045e+03 | 51.79     |
| 15        | -929.5    | 26.48     | 1.426e+03 | 69.21     |
| 16        | -1.082e+0 | 10.0      | 1.873e+03 | 50.0      |


c:\Users\Admin451\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


| 17        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 18        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 19        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 20        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 21        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 22        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 23        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 24        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 25        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 26        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 27        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 28        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 29        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 30        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 31        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 32        | -644.7    | 10.0      | 1.016e+03 | 50.0      |
| 33    